<a href="https://colab.research.google.com/github/viniciusguirra/DashlogisticaSalExpress/blob/main/Sal_express.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Sal Express

## Instalações / Importações

In [ ]:
!pip install gcsfs

In [ ]:
!pip install sweetviz

In [ ]:
!pip install ydata_profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.5/359.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 357.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 11.3 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=b36aafde978a7be51e6d44d00d261fb8dc5a296b54c74590b546e8017bb19952
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin


In [ ]:
# Pacotes de manipulação de dados
import numpy as np
import pandas as pd

# Pacotes gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# Pacotes de análise exploratória
import sweetviz as sv

## Conectores

In [ ]:
# Abertura de bibliotecas de conectores
import os
from google.cloud import storage
# Comando de abertura para Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dados Brutos

In [ ]:
#carregando os dados
file_path = '/content/drive/MyDrive/sal express /AMOSTRA.xlsx'

# Ler o arquivo Excel
df_bruto = pd.read_excel(file_path)

# Mostrar as primeiras linhas do DataFrame
df_bruto.head()

,Origem,Destino,Tipo de Carga,Valor do Frete (R$),Data de Contratação,Data de Entrega,Tempo de Entrega (dias),Forma de Pagamento,Status da Entrega,Custo Adicional (R$),Observações
0,São Paulo,Rio de Janeiro,Eletrônicos,1500,2023-01-05,2023-01-10,5,Cartão de Crédito,Atraso,50,
1,Belo Horizonte,Brasília,Alimentos,800,2023-02-10,2023-02-15,5,Transferência Bancária,Atraso,,
2,Porto Alegre,Curitiba,Produtos Químicos,2000,2023-03-15,2023-03-20,5,Fatura,Atraso,100,
3,Fortaleza,Recife,Roupas,1200,2023-04-20,2023-04-25,5,Transferência Bancária,Atraso,,
4,Manaus,Belém,Livros,1000,2023-05-25,2023-05-30,5,Cartão de Crédito,Atraso,,


In [ ]:
df_bruto.shape

(419, 11)

In [ ]:
df_bruto.head()

,Origem,Destino,Tipo de Carga,Valor do Frete (R$),Data de Contratação,Data de Entrega,Tempo de Entrega (dias),Forma de Pagamento,Status da Entrega,Custo Adicional (R$),Observações
0,São Paulo,Rio de Janeiro,Eletrônicos,1500,2023-01-05,2023-01-10,5,Cartão de Crédito,Atraso,50,
1,Belo Horizonte,Brasília,Alimentos,800,2023-02-10,2023-02-15,5,Transferência Bancária,Atraso,,
2,Porto Alegre,Curitiba,Produtos Químicos,2000,2023-03-15,2023-03-20,5,Fatura,Atraso,100,
3,Fortaleza,Recife,Roupas,1200,2023-04-20,2023-04-25,5,Transferência Bancária,Atraso,,
4,Manaus,Belém,Livros,1000,2023-05-25,2023-05-30,5,Cartão de Crédito,Atraso,,


In [ ]:
df_completa = df_bruto

In [ ]:
def resumetable(df_completa):
    print(f"Dataset Shape: {df_completa.shape}")
    summary = pd.DataFrame(df_completa.dtypes, columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name', 'dtypes']]
    summary['Missing'] = df_completa.isnull().sum().values
    summary['unique'] = df_completa.nunique().values
    summary['First value'] = df_completa.iloc[0].values
    summary['Second value'] = df_completa.iloc[1].values
    return summary

In [ ]:
result = resumetable(df_completa)
result

Dataset Shape: (419, 11)


,Name,dtypes,Missing,unique,First value,Second value
0,Origem,object,0,10,São Paulo,Belo Horizonte
1,Destino,object,0,12,Rio de Janeiro,Brasília
2,Tipo de Carga,object,0,6,Eletrônicos,Alimentos
3,Valor do Frete (R$),int64,0,13,1500,800
4,Data de Contratação,datetime64[ns],0,267,2023-01-05 00:00:00,2023-02-10 00:00:00
5,Data de Entrega,datetime64[ns],0,270,2023-01-10 00:00:00,2023-02-15 00:00:00
6,Tempo de Entrega (dias),int64,0,4,5,5
7,Forma de Pagamento,object,0,3,Cartão de Crédito,Transferência Bancária
8,Status da Entrega,object,0,2,Atraso,Atraso
9,Custo Adicional (R$),object,232,7,50,


Nota-se que os dados são consistentes, apenas algumas valores missing em custo adicional e obersavações, possivelmente por realmente nenhum custo adicional ou observação para ser preenchida. Caso necessário podemos tratá-los após a exportação para Power BI.

## Análise exploratória com Sweetviz

In [ ]:
# Gerar o relatório com sweetviz
report = sv.analyze(df_completa)

# Salvar o relatório em um arquivo HTML
report.show_html('relatorio_sweetviz.html')

                                             |          | [  0%]   00:00 -> (? left)

Report relatorio_sweetviz.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
